## Advanced cryptography course - ISMIN - Advanced Encryption Standard

|               |                          |
|:--------------|:-------------------------|
|Author         |P. GARREAU                |
|Status         |Finished                  |
|Project        |Advanced Cryptography     |

### Table of Content

1. [Types conversions](#1-Types-conversions)  
2. [Elementary transformations](#2-Elementary-transformations)  
    2.1. [AddRoundKey](#21-AddRoundKey)  
    2.2. [SubBytes](#22-SubBytes)  
    2.3. [ShiftRows](#23-ShiftRows)  
    2.4. [MixColumns](#24-MixColumns)  
3. [Key Expansion](#3-Key-Expansion)  
4. [AES Round](#4-AES-Round)  
5. [AES Encoding](#5-AES-Encoding)  

### Useful links

This notebook is based on the following paper anouncing AES : [FIPS 197, Advanced Encryption Standard (AES)](https://nvlpubs.nist.gov/nistpubs/fips/nist.fips.197.pdf)

In [171]:
import crypto
import copy

### 1. Types conversions

This section deals with all types conversions we will need in the AES process. Here are all types needed :

- Text : string
- Bloc : list of length 32, each element is an 8-bits number corresponding to an hexadecimal representation of a character.
- Type State : array of ength 4x4, each element is an 8-bits number.
- Key : a 128-bits hexadecimal number, reprensenting a key used in an AES round.

In [172]:
def nbBlocs(text):
    nbBlocs = 0
    for i in range(0, len(text)):
        if (i % 16 == 0):
            nbBlocs += 1
    return nbBlocs

def textToBloc(text):
    bloc = []
    for i in range(0, len(text)):
        bloc.append(ord(text[i]))
    while (len(bloc) % 16 != 0):
        bloc.append(0)
    return bloc

def keyToText(key):
    text = ""
    for i in range(0, 16):
        text += chr(key >> (128 - 8 * (i + 1)) & 0xFF)
    return text

def blocToTypeState(bloc):
    state_out = [ [ 0 for i in range(0, 4) ] for j in range(0, 4) ]
    for i in range(0, 4):
        for j in range(0, 4):
            state_out[i][j] = bloc[(4 * j + i)]
    return state_out

def typeStateToBloc(state):
    bloc_out = []
    for i in range(0, 16):
        bloc_out.append(state[i % 4][i // 4])
    return bloc_out

def blocToText(bloc):
    text_out = ""
    for i in range(0, 16):
        text_out += format(bloc[i], '02x')
        if (i % 4 == 3):
            text_out += " "
    return text_out

### 2. Elementary transformations

AES encoding uses 4 elementary transformations that we are going to implement one after the other. Here are some details about these transformations:

- AddRoundKey: return the result (type_state) of a xor operation between a input type_state and a key.
- SubBytes: return the result (type_state) of a substitution of each byte of an input type_state thanks to a SBox.
- ShiftRows: return the result (type_state) of a shift of each row of i elements towards the left of an input type_state.
- MixColumns: return the result (type_state) of a matrix multiplication in a Galois Corps of each column of an input type_state and a matrix.

#### 2.1. AddRoundKey

In [173]:
def AddRoundKey(state_in, key):
    state_out = copy.deepcopy(state_in)
    for i in range (0, 4):
        for j in range (0, 4):
            state_out[i][j] = state_in[i][j] ^ key[i][j]
    return state_out

#### 2.2. SubBytes

In [174]:
SBox_hexa = [[0x63,  0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76],
[0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0],
[0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15],
[0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75],
[0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84],
[0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf],
[0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8],
[0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2],
[0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73],
[0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb],
[0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79],
[0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08],
[0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a],
[0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e],
[0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf],
[0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]]

def SubBytes(state_in):
    state_out = copy.deepcopy(state_in)
    for i in range(0, 4):
        for j in range(0, 4):
            indice_ligne = (state_in[i][j] >> 4) & 0xF
            indice_colonne = state_in[i][j] & 0xF
            state_out[i][j] = SBox_hexa[indice_ligne][indice_colonne]
    return state_out

#### 2.3. ShiftRows

In [175]:
def ShiftRows(state_in):
    state_out = copy.deepcopy(state_in)
    for i in range(0, 4):
        for j in range(0, 4):
            state_out[i][(j - i + 4) % 4] = state_in[i][j]
    return state_out

#### 2.4. MixColumns

In [176]:
MatrixMixColumn = [[2, 3, 1, 1], [1, 2, 3, 1], [1, 1, 2, 3], [3, 1, 1, 2]]
Polynome0 = 283

def Mul2(x):
    return x << 1 if x >> 7 == 0 else x << 1 ^ Polynome0

def Mul3(x):
    return (Mul2(x) ^ x) if (Mul2(x) ^ x) >> 8 == 0 else (Mul2(x) ^ x) ^ Polynome0

def MixColumn(state_in):
    state_out = copy.deepcopy(state_in)
    for i in range(0, 4):
        for j in range(0, 4):
            result_item = 0
            for k in range(0, 4):
                if (MatrixMixColumn[i][k] == 2):
                    product = Mul2(state_in[k][j])
                elif (MatrixMixColumn[i][k] == 3):
                    product = Mul3(state_in[k][j])
                else:
                    product = state_in[k][j]
                result_item ^= product
            state_out[i][j] = result_item
    return state_out

### 3. Key Expansion

This section deals with the expansion of a given key. Thanks to a 128-bits input key, this section creates 11 128-bits keys derivated from the original key.

In [177]:
Nk = 4
Nb = 4
Nr = 10
Rcon = [0x00000000, 0x01000000, 0x02000000, 0x04000000, 0x08000000, 0x10000000, 0x20000000, 0x40000000, 0x80000000, 0x1b000000, 0x36000000]

def SubWord(word):
    returned_word = 0x00000000
    for i in range(0, 4):
        indice_ligne = (word >> 8 * i + 4) & 0x0000000F
        indice_colonne = (word >> 8 * i) & 0x0000000F
        returned_word = returned_word ^ (SBox_hexa[indice_ligne][indice_colonne] << 8 * i)
    return returned_word

def RotWord(word):
    returned_word = (word & 0x00FFFFFF) << 8
    return returned_word ^ word >> 24

def KeyExpansion(key):
    w = [ 0x00000000 for _ in range(0, Nb * (Nr + 1)) ]
    temp = 0x00000000

    for i in range(0, Nk):
        w[i] = (key >> (Nk - i - 1) * 32) & 0xFFFFFFFF

    for i in range(Nk, Nb * (Nr + 1)):
        temp = w[i - 1]
        if (i % Nk == 0):
            temp = SubWord(RotWord(temp)) ^ Rcon[i // Nk]
        w[i] = w[i - Nk] ^ temp
    return w

def wordsToKey(words):
    return (words[0] << 96) ^ (words[1] << 64) ^ (words[2] << 32) ^ words[3]

### 4. AES Round

This section perfoms a round of AES encryption. It takes under consideration which round it is to make sure it performs the right transformations.

In [178]:
def AESRound(state_in, i):
    # print("Round " + str(i))
    state_out = copy.deepcopy(state_in)
    if (i == 0):
        # print("[AESRound--] Transformations : AddRoundKey")
        state_out = AddRoundKey(state_in, keys_type_state[i])
    elif (i == 10):
        # print("[AESRound--] Transformations : AddRoundKey / ShiftRows / SubBytes")
        state_out = AddRoundKey(ShiftRows(SubBytes(state_in)), keys_type_state[i])
    else:
        # print("[AESRound--] Transformations : AddRoundKey / MixColumn / ShiftRows / SubBytes")
        # print(blocToText(typeStateToBloc(SubBytes(state_in))))
        # print(blocToText(typeStateToBloc(ShiftRows(SubBytes(state_in)))))
        # print(blocToText(typeStateToBloc(MixColumn(ShiftRows(SubBytes(state_in))))))

        # print(blocToText(typeStateToBloc(keys_type_state[i])))
        state_out = AddRoundKey(MixColumn(ShiftRows(SubBytes(state_in))), keys_type_state[i])
    # print(blocToText(typeStateToBloc(state_out)))
    return state_out

### 5. AES Encoding

In this section, we encode each 128-bits bloc one by one.

In [179]:
def AESBloc(state_in, numBloc):
    print("[AESBLOC---] Starting encoding bloc " + str(numBloc))
    state_out = copy.deepcopy(state_in)
    for i in range(0, 11):
        state_out = AESRound(state_out, i)
    print("[AESBLOC---] Bloc " + str(numBloc) + " encoded")
    return state_out
    
def AES(textToCipher, key):
    # Initialization
    print("[AES-------] Starting encoding text")
    nb_blocs = nbBlocs(textToCipher)
    blocs_in = []
    states_in = []
    states_out = []
    blocs_out = []
    text_out = ""

    # Key expansion
    keys = KeyExpansion(key)
    keys_array = [ 0x00000000 for _ in range(0, Nr + 1) ]
    keys_type_state = [ [ [ 0 for _ in range(0, 4) ] for _ in range(0, 4) ] for _ in range(0, Nr + 1) ]
    for i in range(0, Nr + 1):
        keys_array[i] = wordsToKey(keys[4 * i:4 * (i + 1)])
        keys_type_state[i] = blocToTypeState(textToBloc(keyToText(keys_array[i])))

    # AES process
    for i in range(0, nb_blocs):
        blocs_in.append(textToBloc(textToCipher[16 * i : 16 * (i + 1)]))
        states_in.append(blocToTypeState(blocs_in[i]))
        states_out.append(AESBloc(states_in[i], i))
        blocs_out.append(typeStateToBloc(states_out[i]))
        for j in range(0, 16):
            text_out += format(blocs_out[i][j], '02x')
    print("[AES-------] Text encoded")
    return text_out

text_to_cipher = "j'aime ce module car il est trop bien"
key = 0x2b7e151628aed2a6abf7158809cf4f3c

print("Text to cipher : " + text_to_cipher)
cipher_text = AES(text_to_cipher, key)
print("Cipher text : " + str(cipher_text))

Text to cipher : j'aime ce module car il est trop bien
[AES-------] Starting encoding text
[AESBLOC---] Starting encoding bloc 0
[AESBLOC---] Bloc 0 encoded
[AESBLOC---] Starting encoding bloc 1
[AESBLOC---] Bloc 1 encoded
[AESBLOC---] Starting encoding bloc 2
[AESBLOC---] Bloc 2 encoded
[AES-------] Text encoded
Cipher text : 92cd969942f78d7a6a70509961be177f00ee6dd6690c7dec589e8475ed20b19ade52ea18764478d0dab23a057495b1b4
